In [3]:
"""
Created on Mon Feb 10 16:36:55 2020
No stealy 
@author: ecouv
"""

from google.cloud import speech_v1
from scipy.io import wavfile
from scipy import signal
import matplotlib.pyplot as plt
import math
import numpy as np
import time
import speech_recognition as sr
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="JSON/My First Project-1534988de9b5.json"

storage_uri = 'gs://ringr_audio/venv/RawAudio/case2.wav'

# Make a sound player function that plays array "x" with a sample rate "rate", and labels it with "label"
def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))
        
    
def sample_long_running_recognize(storage_uri):
    """
    Print start and end time of each word spoken in audio file from Cloud Storage

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.flac'

    # When enabled, the first result returned by the API will include a list
    # of words and the start and end time offsets (timestamps) for those words.
    enable_word_time_offsets = True

    # The language of the supplied audio
    language_code = "en-US"
    config = {
        "enable_word_time_offsets": enable_word_time_offsets,
        "language_code": language_code,
    }
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    # The first result includes start and end time word offsets
    result = response.results[0]
    # First alternative is the most probable result
    alternative = result.alternatives[0]
    print(u"Transcript: {}".format(alternative.transcript))
    # Print the start and end time of each word
    for word in alternative.words:
        print(u"Word: {}".format(word.word))
        print(
            u"Start time: {} seconds {} nanos".format(
                word.start_time.seconds, word.start_time.nanos
            )
        )
        print(
            u"End time: {} seconds {} nanos".format(
                word.end_time.seconds, word.end_time.nanos
            )
        )    
  
# just want sampling rate for replays

sample_long_running_recognize(storage_uri)

#########  CASE 1 #########




Waiting for operation to complete...
Transcript: calculate top priority on Mini Bike Fix
Word: calculate
Start time: 0 seconds 0 nanos
End time: 0 seconds 600000000 nanos
Word: top
Start time: 0 seconds 600000000 nanos
End time: 0 seconds 900000000 nanos
Word: priority
Start time: 0 seconds 900000000 nanos
End time: 1 seconds 100000000 nanos
Word: on
Start time: 1 seconds 100000000 nanos
End time: 1 seconds 400000000 nanos
Word: Mini
Start time: 1 seconds 400000000 nanos
End time: 1 seconds 600000000 nanos
Word: Bike
Start time: 1 seconds 600000000 nanos
End time: 1 seconds 900000000 nanos
Word: Fix
Start time: 1 seconds 900000000 nanos
End time: 2 seconds 200000000 nanos
